# Phase 2

### Data transformation

### 1. Read the data from the staging area and find outliers, null or empty data, and more.

In [1]:
import os
from dotenv import load_dotenv
import psycopg2
from sqlalchemy import create_engine, text
import pandas as pd

# Load environment variables
load_dotenv()

db_staging_user = os.getenv('DB_STAGING_USER')
db_staging_password = os.getenv('DB_STAGING_PASSWORD')
db_staging_host = os.getenv('DB_STAGING_HOST')
db_staging_port = int(os.getenv('DB_STAGING_PORT', 5432))
db_staging_name = os.getenv('DB_STAGING_NAME')

resources_path = os.getenv('RESOURCES_PATH')
mascotas_propietarios_filename = 'Mascotas_Propietarios_despensaAnimal_Generated.csv'
propietarios_transacciones_filename = 'Propietarios_Transacciones_despensaAnimal_Generated.csv'

if db_staging_user is None:
    raise ValueError('DB_STAGING_USER is not set')
if db_staging_password is None:
    raise ValueError('DB_STAGING_PASSWORD is not set')
if db_staging_host is None:
    raise ValueError('DB_STAGING_HOST is not set')
if db_staging_port is None:
    raise ValueError('DB_STAGING_PORT is not set')
if db_staging_name is None:
    raise ValueError('DB_STAGING_NAME is not set')
if resources_path is None:
    raise ValueError('RESOURCES_PATH is not set')

connStaging = psycopg2.connect(
    dbname=db_staging_name,
    user=db_staging_user,
    password=db_staging_password,
    host=db_staging_host,
    port=db_staging_port
)
connStaging.autocommit = True   

#### 1.1 Create the database engine for the staging data

In [2]:
db_staging_engine = create_engine(f'postgresql://{db_staging_user}:{db_staging_password}@{db_staging_host}:{db_staging_port}/{db_staging_name}')

df_mascotas_propietarios_staging = pd.read_sql('SELECT * FROM mascotas_propietarios_staging;', db_staging_engine)
df_mascotas_propietarios_staging

,nombre_mascota,raza,peso,fecha_nacimiento,sexo,temperamento,numero_carnet,estado_reproductivo,numero_partos,color,...,ciudad,direccion,telefono,whatsapp,email,tipo_documento,numero_documento,profesion,estado,notificaciones_whatsapp
0,THANOS URREGO,Poodle,None,2022-09-01,Macho,social,1631,Entero,None,None,...,Cali,CRA100 #28-68 VALLE DEL LILI MADEIRO TORRE 1 ...,None,3207201137,lizethurrego1990@gmail.com,CC,1036637677,None,Activo,Activo
1,TINENK ORDONEZ,Shit-zu,None,2017-02-17,Hembra,social,1650,None,None,None,...,Santiago de Cali,cra 85 c # 33- 40 casa 54,3058147632,3157929392,pa_ordonez@hotmail.com,CC,66987417,None,Activo,Activo
2,SIMON VACCA,PUG,None,2021-08-01,Macho,social,1184,None,None,None,...,None,Cra98B #45-200 SAN MIGUEL,None,3234191060,None,CC,None,None,Activo,Activo
3,SUSSY PUERTA,YORK SHIRE TERRIER,3.6,2016-10-07,Hembra,social,1359,ENTERA,NINGUNO,None,...,None,Cra. 98b #34-53 GUADALQUIVIR CASA 64,3134024437,3146096191,None,CC,1006107262,None,Activo,Activo
4,EEVEE ANACONA,Poodle,None,2017-09-03,Hembra,social,1632,None,None,None,...,Cali,"Calle 45#98B-65 Apto 403, torre 8",None,3216066041,jazmin.dag@gmail.com,CC,1061750508,None,Activo,Activo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11994,Jennifer,Golden Retriever,46.1,2024-07-17,Hembra,Agresivo,FB13AD2F,Esterilizado,1,Negro,...,Cali,2256 Daniel Camp,3139761188,None,jacob96@example.com,NIT,176580896,Panadero,Inactivo,Si
11995,Timothy,German Shepherd,41.79,2014-03-10,Macho,Agresivo,2361BF30,No Esterilizado,0,Blanco,...,Cali,194 Kimberly Tunnel Apt. 031,3184965612,None,shelleyparker@example.com,NIT,130286470,Ingeniero,Activo,No
11996,Marcus,Golden Retriever,32.76,2014-09-16,Hembra,Sociable,85D44DE5,Esterilizado,0,Marrón,...,Cali,237 Castro Meadow,3168512890,None,ojohnson@example.org,CE,194581954,Ingeniero,Activo,No
11997,James,Poodle,35.41,2005-08-23,Hembra,Sociable,4C95A084,No Esterilizado,5,Gris,...,Cali,6386 Miller Place,3201628333,None,jasminecarr@example.com,NIT,174849231,Arquitecta,Inactivo,No


### 2. Remove data that not add value to the transaccions scenario

1.	comentarios_fallecimiento → Not useful for purchases or analysis of services.
2.	motivo_fallecimiento → Not related to transactions.
3.	fecha_fallecimiento → If the pet has died, it will no longer generate new purchases.
4.	numero_carnet → Not relevant to link purchases with clients.
5.	estado_reproductivo → Does not influence the purchase of services.
6.	numero_partos → Does not seem to affect the purchase of veterinary services.
7.	profesion → Not directly related to service purchases.
8.	notificaciones_whatsapp → Not used to analyze transactions.

In [3]:
# Function to drop unnecessary columns
def drop_unnecessary_columns(df: pd.DataFrame) -> pd.DataFrame:
    columns_to_drop = [
        'comentarios_fallecimiento',
        'motivo_fallecimiento',
        'fecha_fallecimiento',
        'numero_carnet',
        'estado_reproductivo',
        'numero_partos',
        'profesion',
        'notificaciones_whatsapp'
    ]

    for col in columns_to_drop:
        if col in df.columns:
            print(f'Removing column: {col}')
            df.drop(columns=[col], inplace=True)
        else:
            print(f'Warning: Column {col} not found in the dataset.')
    
    print(f'Columns removed. New shape: {df.shape}')
    return df

# Apply the function
df_mascotas_propietarios_cleaned = drop_unnecessary_columns(df_mascotas_propietarios_staging)
df_mascotas_propietarios_cleaned

Removing column: comentarios_fallecimiento
Removing column: motivo_fallecimiento
Removing column: fecha_fallecimiento
Removing column: numero_carnet
Removing column: estado_reproductivo
Removing column: numero_partos
Removing column: profesion
Removing column: notificaciones_whatsapp
Columns removed. New shape: (11999, 16)


,nombre_mascota,raza,peso,fecha_nacimiento,sexo,temperamento,color,nombre_propietario,ciudad,direccion,telefono,whatsapp,email,tipo_documento,numero_documento,estado
0,THANOS URREGO,Poodle,None,2022-09-01,Macho,social,None,LIZETH URREGO,Cali,CRA100 #28-68 VALLE DEL LILI MADEIRO TORRE 1 ...,None,3207201137,lizethurrego1990@gmail.com,CC,1036637677,Activo
1,TINENK ORDONEZ,Shit-zu,None,2017-02-17,Hembra,social,None,PAOLA ORDONEZ,Santiago de Cali,cra 85 c # 33- 40 casa 54,3058147632,3157929392,pa_ordonez@hotmail.com,CC,66987417,Activo
2,SIMON VACCA,PUG,None,2021-08-01,Macho,social,None,ANDREA VACCA,None,Cra98B #45-200 SAN MIGUEL,None,3234191060,None,CC,None,Activo
3,SUSSY PUERTA,YORK SHIRE TERRIER,3.6,2016-10-07,Hembra,social,None,MARIA CAMILA PUERTA,None,Cra. 98b #34-53 GUADALQUIVIR CASA 64,3134024437,3146096191,None,CC,1006107262,Activo
4,EEVEE ANACONA,Poodle,None,2017-09-03,Hembra,social,None,YAZMIN ANACONA,Cali,"Calle 45#98B-65 Apto 403, torre 8",None,3216066041,jazmin.dag@gmail.com,CC,1061750508,Activo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11994,Jennifer,Golden Retriever,46.1,2024-07-17,Hembra,Agresivo,Negro,Steven Keller,Cali,2256 Daniel Camp,3139761188,None,jacob96@example.com,NIT,176580896,Inactivo
11995,Timothy,German Shepherd,41.79,2014-03-10,Macho,Agresivo,Blanco,Christopher Clark,Cali,194 Kimberly Tunnel Apt. 031,3184965612,None,shelleyparker@example.com,NIT,130286470,Activo
11996,Marcus,Golden Retriever,32.76,2014-09-16,Hembra,Sociable,Marrón,Lisa Reyes,Cali,237 Castro Meadow,3168512890,None,ojohnson@example.org,CE,194581954,Activo
11997,James,Poodle,35.41,2005-08-23,Hembra,Sociable,Gris,Paul White,Cali,6386 Miller Place,3201628333,None,jasminecarr@example.com,NIT,174849231,Inactivo


### 2.1 Analyze the field: ciudad and its different values

In [4]:
df_ciudad = df_mascotas_propietarios_cleaned.groupby("ciudad").size().reset_index(name="total_registros")
df_ciudad

,ciudad,total_registros
0,CAI,1
1,CALI,1692
2,CALUI,1
3,CRA 64 # 14-24,1
4,Cali,9259
5,Calle 60 b # 119 - 47 Torre 2 apto 602 Laurel,1
6,EEUU,5
7,JAMUNDI,8
8,Santiago de Cali,555
9,cali,54


We can see that the field: 'ciudad' doesn't have the same value and, also has information relates with countries, others towns and addresses, we must clean this data.

In [5]:
# Update all values in 'ciudad' column to 'Cali'
df_mascotas_propietarios_cleaned["ciudad"] = "Cali"

df_ciudad = df_mascotas_propietarios_cleaned.groupby("ciudad").size().reset_index(name="total_registros")
df_ciudad

,ciudad,total_registros
0,Cali,11999
